# piont cloud separator

In [1]:
import numpy as np
import tensorflow as tf
from pyntcloud import PyntCloud

In [2]:
def find_ranges(point_cloud):

    x_max = x_min = point_cloud[0][0]
    y_max = y_min = point_cloud[0][1]
    z_max = z_min = point_cloud[0][2]
    
    for ind, coor in enumerate(point_cloud):
        if coor[0] > x_max:
            x_max = coor[0]
        if coor[0] < x_min:
            x_min = coor[0]
        if coor[1] > y_max:
            y_max = coor[1]
        if coor[1] < y_min:
            y_min = coor[1]
        if coor[2] > z_max:
            z_max = coor[2]
        if coor[2] < z_min:
            z_min = coor[2]
            
    return ((x_min, x_max), (y_min, y_max), (z_min, z_max))


def find_cube_length(t_range, overall_size):
    length = (t_range[1] - t_range[0])/overall_size
    return length

def get_all_length(point_cloud, overall_size, xyz_ranges):
    
    x_length = find_cube_length(xyz_ranges[0], overall_size[0])
    y_length = find_cube_length(xyz_ranges[1], overall_size[1])
    z_length = find_cube_length(xyz_ranges[2], overall_size[2])
    
    return (x_length, y_length, z_length)
    
def find_target_area_range(point_cloud, target_area, xyz_length, xyz_ranges):
    
    target_x_range_max = xyz_ranges[0][1] - target_area[0][0]*xyz_length[0]
    target_x_range_min = xyz_ranges[0][1] - target_area[0][1]*xyz_length[0]
    target_y_range_max = xyz_ranges[1][1] - target_area[1][0]*xyz_length[1]
    target_y_range_min = xyz_ranges[1][1] - target_area[1][1]*xyz_length[1]
    target_z_range_max = xyz_ranges[2][1] - target_area[2][0]*xyz_length[2]
    target_z_range_min = xyz_ranges[2][1] - target_area[2][1]*xyz_length[2]
    
    return (target_x_range_min, target_x_range_max), (target_y_range_min, target_y_range_max), (target_z_range_min, target_z_range_max)


def segment_points(point_cloud, target_ranges):
    """
    Recives orig point cloud data and a tuple of boundary tuples like:
        ((x_min, x_max), (y_min, y_max), (z_min, z_max))
    """
    
    target_x_range, target_y_range, target_z_range = target_ranges[0], target_ranges[1], target_ranges[2]
    
    points = []
    for ind, coor in enumerate(point_cloud):
        if coor[0] >= target_x_range[0] and coor[0] <= target_x_range[1]:
            if coor[1] >= target_y_range[0] and coor[1] <= target_y_range[1]:
                if coor[2] >= target_z_range[0] and coor[2] <= target_z_range[1]:
                    points.append(coor)
                    
    return np.asarray(points)

In [3]:
def traverse_point(point_cloud, cube_size=(3,3,3), overall_size=[32,32,32], stride=(1,1,1), filter_percentage=None):
    """
    It will return the target segment of point cloud.
    """
    threshold = 0
    
    if filter_percentage is not None:
        threshold = filter_percentage*len(point_cloud)
    
    xyz_range = find_ranges(point_cloud)
    xyz_length = get_all_length(point_cloud, overall_size, xyz_range)

    segmentations = []
    areas = []
    
    x_num = int((overall_size[0] - cube_size[0]) / stride[0] + 1)
    y_num = int((overall_size[1] - cube_size[1]) / stride[1] + 1)
    z_num = int((overall_size[2] - cube_size[2]) / stride[2] + 1)
    
    for _x in range(x_num):
        for _y in range(y_num):
            for _z in range(z_num):
                target_area = ((_x*stride[0], _x*stride[0] + cube_size[0]), (_y*stride[1], _y*stride[1] + cube_size[1]), (_z*stride[2], _z*stride[2] + cube_size[2]))
    
                target_x_range, target_y_range, target_z_range = find_target_area_range(point_cloud, target_area, xyz_length, xyz_range)
                
                segmented_point_cloud = segment_points(point_cloud, (target_x_range, target_y_range, target_z_range))
                
                if len(segmented_point_cloud) > threshold:
                    
                    segmentations.append(segmented_point_cloud)
                    
                    areas.append(target_area)
                    
                    
    if overall_size[0]%cube_size[0] is not 0 or overall_size[1]%cube_size[1] is not 0 or overall_size[2]%cube_size[2] is not 0:
        
        target_area = ((overall_size[0] - cube_size[0], overall_size[0]), (overall_size[1] - cube_size[1], overall_size[1]), (overall_size[2] - cube_size[2], overall_size[2]))
        
        target_x_range, target_y_range, target_z_range = find_target_area_range(point_cloud, target_area, xyz_length, xyz_range)

        segmented_point_cloud = segment_points(point_cloud, (target_x_range, target_y_range, target_z_range))
            
        if len(segmented_point_cloud) > threshold:
            
            segmentations.append(segmented_point_cloud)   
            
            areas.append(target_area)
    
    return {'data': segmentations, 'area': areas}

In [4]:
import os
my_point_cloud = PyntCloud.from_file(os.path.join(os.getcwd(), 'ttt.pts'), sep=" ", header=0, names=["x","y","z"])

In [5]:
segs = traverse_point(my_point_cloud.xyz, cube_size=(3,3,3), overall_size=[13,13,13], stride=(3,3,3))

In [6]:
def voxelize3D(pts, dim=[1,1,1]):
    """
    pts: receives .pts cloud point data. 2D array, arbitary sized X,Y,Z pairs. (We will only take x,y,z into account for now)
    dim: dimensioin of output voxelized data
    
    This function will locate the grid cube and calculate the density of each cube.
    The output will be normalized values.
    """
    assert(pts.shape[1]>=3), "pts file should contain at least x,y,z coordinate"
    assert(len(dim)==3), "Please provide 3-d grid size like [32,32,32]"
    
    if len(pts) > 1:
        # move all the axis to positive area.
        minimum_val = [pts[0][0], pts[0][1], pts[0][2]]

        # find the smallest 
        for pair in pts:
            if pair[0] < minimum_val[0]:
                minimum_val[0] = pair[0]
            if pair[1] < minimum_val[1]:
                minimum_val[1] = pair[1]
            if pair[2] < minimum_val[2]:
                minimum_val[2] = pair[2]

        # move it to first quadrant 
        rectified_pts = np.empty(pts.shape)
        for index, pair in enumerate(pts):
            point = np.zeros(3)
            point[0] = pair[0] - minimum_val[0]
            point[1] = pair[1] - minimum_val[1]
            point[2] = pair[2] - minimum_val[2]
            rectified_pts[index] = point

        # biggest value in each axis 
        maximum_val = pts[0][0]

        for pair in rectified_pts:
            for val in pair:
                if val > maximum_val:
                    maximum_val = val

        # normalize all the axises to (0,1)
        normalized_pts = rectified_pts/maximum_val
    
    else:
        # in case there is just one point
        normalized_pts = pts
    
    x_grid_length = 1/dim[0]
    y_grid_length = 1/dim[1]
    z_grid_length = 1/dim[2]
    
    output = np.zeros((dim[0],dim[1],dim[2]))
    
    epsilon = 0.000000000001 # we will have at least a 1.0 value which will exceed the index of grid
    # we can use a relativly small value to escape that to fit our data
    
    max_volume_size = 0
    
    for pair in normalized_pts:
        x_loc = int(pair[0]/(x_grid_length + epsilon))
        y_loc = int(pair[1]/(y_grid_length + epsilon))
        z_loc = int(pair[2]/(z_grid_length + epsilon))
        if output[x_loc, y_loc, z_loc] is None:
            output[x_loc, y_loc, z_loc] = 1
        else:
            output[x_loc, y_loc, z_loc] += 1
        
        if output[x_loc, y_loc, z_loc] > max_volume_size:
            max_volume_size = output[x_loc, y_loc, z_loc]
    
    output = output/max_volume_size    
            
    return output

In [7]:
print(len(my_point_cloud.xyz), len(segs['data'][0]))
print(len(segs['area']))
vox_segs = []
whole_vox = voxelize3D(my_point_cloud.xyz, dim=[32,32,32])
whole_vox = whole_vox.reshape(whole_vox.shape + (1,))

for idx, value in enumerate(segs['data']):
    vox = voxelize3D(value, dim=[32,32,32])
    vox_chan = np.array(vox).reshape(vox.shape + (1,))
    vox_segs.append(vox_chan)

5167 280
22


In [23]:
my_point_cloud.plot()

In [8]:
import h5py
import os
import tensorflow as tf

# Create hdf5
hdf5_path = os.path.join(os.getcwd(), 'h5dataset', 'big_shuffled_data.h5')
hdf5_file = h5py.File(hdf5_path, mode='r')

b_data = hdf5_file.get('voxels')
b_labels = hdf5_file.get('labels')
b_label_ref = hdf5_file.get('label_ref')

OSError: Unable to open file (unable to open file: name = '/Users/shijian/git/3D-CNN/3d_pointcloud/h5dataset/big_shuffled_data.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [104]:
def count_v(probs, cube_labels):
    li = dict()
    cubes = dict()
    for _, val in enumerate(probs):
        _max = np.argmax(val)
        if _max not in li:
            li.update({_max:1})
            m_li = list()
            m_li.append(cube_labels[_])
            cubes.update({_max: m_li})
        else:
            li.update({_max: li[_max]+1})
            m_li = cubes[_max]
            m_li.append(cube_labels[_])
            cubes.update({_max: m_li})
    return li, cubes

In [150]:
# one hot indexes

import os
model_path = os.path.join(os.getcwd(), 'trained_model', 'model-2')

device_name = '/gpu:1' 

config = tf.ConfigProto(allow_soft_placement = True)
with tf.Session(graph=tf.Graph(), config=config) as sess:
    with tf.device(device_name):

        saver = tf.train.import_meta_graph(model_path + ".meta")
        saver.restore(sess, model_path)

        graph = tf.get_default_graph()
        x_input = graph.get_tensor_by_name('inputs/x_input:0')
        y_input = graph.get_tensor_by_name('inputs/y_input:0')
        pred = graph.get_collection('logits')
        accuracy = graph.get_tensor_by_name('acc:0')

#         y, prediction = sess.run([accuracy, pred], feed_dict={x_input: b_data[30:60], y_input: b_labels[30:60]})
#         print(tf.nn.softmax(tf.constant(prediction)).eval())
#         print(y)
        probs = []
        ys = []
        for _, val in enumerate(vox_segs):
            y = sess.run(pred, feed_dict={x_input: [val]})
            ys.append(tf.argmax(np.asarray(y[0]), axis=1).eval())
            probs.append(tf.nn.softmax(np.asarray(y[0])).eval())

INFO:tensorflow:Restoring parameters from C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2


INFO:tensorflow:Restoring parameters from C:\Users\212606295\Desktop\3D-CNN\3d_pointcloud\trained_model\model-2


In [12]:
def to_density(occ_list, upper=[32,32,32]):
    res = np.zeros((upper[0], upper[1], upper[2]))
    for occ_row in occ_list:
        for _x in range(occ_row[0][1] - occ_row[0][0]):
            for _y in range(occ_row[1][1] - occ_row[1][0]):
                for _z in range(occ_row[2][1] - occ_row[2][0]):
                    res[occ_row[0][0]+_x][occ_row[1][0]+_y][occ_row[2][0]+_z] = 1
    return res

In [151]:
argmax, occ = count_v(probs, segs['area'])
print(argmax)

{1: 11, 4: 7, 2: 1, 0: 2, 3: 1}


In [11]:
def write_to_file(xyz, filename='xyz'):
    import os
    file = open(os.path.join(os.getcwd(), filename + ".pts"), "w") 
    
    for point in xyz:
        st = ""
        for item in point:
            st += str(item) + " "
        file.write(st.strip() + "\n")

    file.close() 
    
# write_to_file(segmented_point_cloud)

In [194]:
# %load ./voxel_grid_plot.py
import os
import shutil

import numpy as np
import pandas as pd

from IPython.display import IFrame
from matplotlib import pyplot as plt
from pyntcloud import PyntCloud

def plot_voxel_points(voxelgrid,
                   point_cloud,
                   output_name=None,
                   cmap="Oranges",
                   axis=True,
                   width=800,
                   height=600):

    # For Voxel Grid
    scaled_shape = voxelgrid.shape

    vector = voxelgrid
    points = np.argwhere(vector)

    s_m = plt.cm.ScalarMappable(cmap=cmap)
    rgb = s_m.to_rgba(vector.reshape(-1)[vector.reshape(-1) > 0])

    camera_position = points.max(0) + abs(points.max(0))

    look = points.mean(0)

    if axis:
        axis_size = points.ptp() * 1.5
    else:
        axis_size = 0
    
    # For point cloud
    filename = 'pyntcloud_plot'
    
    path = os.path.join(os.getcwd(), filename + ".pts")
    # Fit point cloud into Voxel Grid
    (x_min, x_max), (y_min, y_max), (z_min, z_max) = find_ranges(point_cloud)
#     pts_camera_position = (point_cloud.max(0) + abs(point_cloud.max(0))).tolist()
    new_point_cloud = np.empty(point_cloud.shape)
    for idx, row in enumerate(point_cloud):
        _r = np.empty(row.shape)
        _r[0] = (row[0] - x_min)/(x_max - x_min)*scaled_shape[0]
        _r[1] = (row[1] - y_min)/(y_max - y_min)*scaled_shape[1]
        _r[2] = (row[2] - z_min)/(z_max - z_min)*scaled_shape[2]
        new_point_cloud[idx] = _r
    point_cloud = new_point_cloud
    
    # Orange by default
    colors = np.repeat([[255, 125, 0]], point_cloud.shape[0], axis=0)
    colors = colors.astype(np.uint8)
    points_df = pd.DataFrame(point_cloud, columns=["x", "y", "z"])
    for n, i in enumerate(["red", "green", "blue"]):
        points_df[i] = colors[:, n]
    cloud = PyntCloud(points_df)
    
    ply_gen = cloud.to_file("{}.ply".format(filename), also_save=["mesh"])
    
#     look_at = cloud.xyz.mean(0).tolist()
    

    placeholders = {}

    placeholders["POINTS_X_PLACEHOLDER"] = points[:, 0].tolist()
    placeholders["POINTS_Y_PLACEHOLDER"] = points[:, 1].tolist()
    placeholders["POINTS_Z_PLACEHOLDER"] = points[:, 2].tolist()

    placeholders["R_PLACEHOLDER"] = rgb[:, 0].tolist()
    placeholders["G_PLACEHOLDER"] = rgb[:, 1].tolist()
    placeholders["B_PLACEHOLDER"] = rgb[:, 2].tolist()

    placeholders["S_x_PLACEHOLDER"] = 2
    placeholders["S_y_PLACEHOLDER"] = 2
    placeholders["S_z_PLACEHOLDER"] = 2

    placeholders["CAMERA_X_PLACEHOLDER"] = camera_position[0]
    placeholders["CAMERA_Y_PLACEHOLDER"] = camera_position[1]
    placeholders["CAMERA_Z_PLACEHOLDER"] = camera_position[2]

    placeholders["LOOK_X_PLACEHOLDER"] = look[0]
    placeholders["LOOK_Y_PLACEHOLDER"] = look[1]
    placeholders["LOOK_Z_PLACEHOLDER"] = look[2]

    placeholders["AXIS_SIZE_PLACEHOLDER"] = axis_size

    placeholders["N_VOXELS_PLACEHOLDER"] = sum(vector.reshape(-1) > 0)
    
    placeholders["FILENAME_PLACEHOLDER"] = "\"" + filename + "\""
    placeholders["POINT_SIZE_PLACEHOLDER"] = 0.3

    
    if output_name is None:
        output_name = "plotVG.html"
    
    BASE_PATH = os.getcwd()
    src = "{}/{}".format(BASE_PATH, "obj_detection_plot.html")
    dst = "{}/{}".format(os.getcwd(), output_name)
    point_size=0.001
    
    with open(src, "r") as inp, open(dst, "w") as out:
        for line in inp:
            for key, val in placeholders.items():
                if key in line:
                    line = line.replace(key, str(val))
            out.write(line)

    return IFrame(output_name, width=width, height=height)

In [195]:
# to_density(occ[4], upper=[13,13,13])
x = np.ones((13,13,13))
plot_voxel_points(x, my_point_cloud.xyz)

# produce multi data

In [145]:
my_point_cloud_a = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'airplane', 'points', '1a74b169a76e651ebc0909d98a1ff2b4.pts'), sep=" ", header=0, names=["x","y","z"])
my_point_cloud_b = PyntCloud.from_file(os.path.join(os.getcwd(), 'PartAnnotation', 'lamp', 'points', '1a9c1cbf1ca9ca24274623f5a5d0bcdc.pts'), sep=" ", header=0, names=["x","y","z"])

print(type(my_point_cloud_a.xyz))
c = np.concatenate((my_point_cloud_a.xyz, my_point_cloud_b.xyz + 0.5))
print(c.shape)

write_to_file(c, filename="ttt")

<class 'numpy.ndarray'>
(5168, 3)


In [161]:
PyntCloud.from_file(os.path.join(os.getcwd(), 'sq.pts'), sep=" ", header=0, names=["x","y","z"]).plot(point_size=0.01)

# Testing

In [135]:
def find_ranges(point_cloud):

    x_max = x_min = point_cloud[0][0]
    y_max = y_min = point_cloud[0][1]
    z_max = z_min = point_cloud[0][2]
    
    for ind, coor in enumerate(point_cloud):
        if coor[0] > x_max:
            x_max = coor[0]
        if coor[0] < x_min:
            x_min = coor[0]
        if coor[1] > y_max:
            y_max = coor[1]
        if coor[1] < y_min:
            y_min = coor[1]
        if coor[2] > z_max:
            z_max = coor[2]
        if coor[2] < z_min:
            z_min = coor[2]
            
    return ((x_min, x_max), (y_min, y_max), (z_min, z_max))


def find_cube_length(t_range, overall_size):
    length = (t_range[1] - t_range[0])/overall_size
    return length

def get_all_length(point_cloud, overall_size, xyz_ranges):
    
    x_length = find_cube_length(xyz_ranges[0], overall_size[0])
    y_length = find_cube_length(xyz_ranges[1], overall_size[1])
    z_length = find_cube_length(xyz_ranges[2], overall_size[2])
    
    return (x_length, y_length, z_length)
    
def find_target_area_range(point_cloud, target_area, xyz_length, xyz_ranges):
    
    target_x_range_max = xyz_ranges[0][1] - target_area[0][0]*xyz_length[0]
    target_x_range_min = xyz_ranges[0][1] - target_area[0][1]*xyz_length[0]
    target_y_range_max = xyz_ranges[1][1] - target_area[1][0]*xyz_length[1]
    target_y_range_min = xyz_ranges[1][1] - target_area[1][1]*xyz_length[1]
    target_z_range_max = xyz_ranges[2][1] - target_area[2][0]*xyz_length[2]
    target_z_range_min = xyz_ranges[2][1] - target_area[2][1]*xyz_length[2]
    
    return (target_x_range_min, target_x_range_max), (target_y_range_min, target_y_range_max), (target_z_range_min, target_z_range_max)


def segment_points(point_cloud, target_ranges):
    """
    Recives orig point cloud data and a tuple of boundary tuples like:
        ((x_min, x_max), (y_min, y_max), (z_min, z_max))
    """
    
    target_x_range, target_y_range, target_z_range = target_ranges[0], target_ranges[1], target_ranges[2]
    
    points = []
    for ind, coor in enumerate(point_cloud):
        if coor[0] >= target_x_range[0] and coor[0] <= target_x_range[1]:
            if coor[1] >= target_y_range[0] and coor[1] <= target_y_range[1]:
                if coor[2] >= target_z_range[0] and coor[2] <= target_z_range[1]:
                    points.append(coor)
                    
    return np.asarray(points)

In [ ]:
def diagonal_cubes(point_cloud, overall_size=(16,16,16)):
    
    xyz_range = find_ranges(point_cloud)
    xyz_length = get_all_length(point_cloud, overall_size, xyz_range)
    
    